In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime as dt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [51]:
targets = pd.read_csv(pd_data_base_path + 'Targets.csv')
transactions = pd.read_csv(pd_data_base_path + 'PD 2023 Wk 1 Input.csv')

In [52]:
targets.head()

,Online or In-Person,Q1,Q2,Q3,Q4
0,Online,72500,70000,60000,60000
1,In-Person,75000,70000,70000,60000


In [53]:
transactions.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
0,DTB-716-679-576,1448,100001,2,20/03/2023 00:00:00
1,DS-795-814-303,7839,100001,2,15/11/2023 00:00:00
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00
3,DS-367-545-264,7957,100007,2,18/08/2023 00:00:00
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00


In [54]:
transactions['Online or In-Person'] = transactions['Online or In-Person'].apply(lambda x: 'Online' if x==1 else 'In-Person')

In [55]:
transactions.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
0,DTB-716-679-576,1448,100001,In-Person,20/03/2023 00:00:00
1,DS-795-814-303,7839,100001,In-Person,15/11/2023 00:00:00
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00
3,DS-367-545-264,7957,100007,In-Person,18/08/2023 00:00:00
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00


In [56]:
transactions['Bank Code'] = transactions['Transaction Code'].apply(lambda x: x.split('-')[0])
transactions = transactions[transactions['Bank Code'] == 'DSB']

In [57]:
transactions.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Bank Code
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00,DSB
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00,DSB
5,DSB-448-546-348,4525,100009,Online,27/05/2023 00:00:00,DSB
11,DSB-422-218-322,118,100010,Online,12/05/2023 00:00:00,DSB
12,DSB-669-227-170,830,100001,Online,15/04/2023 00:00:00,DSB


In [58]:
transactions['Transaction Date'] = transactions['Transaction Date'].astype('datetime64[ns]').apply(lambda x: 'Q' + str(math.ceil((int(x.strftime("%m")) - 1)//3)+1))

In [59]:
transactions.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,Bank Code
2,DSB-807-592-406,5520,100005,Online,Q3,DSB
4,DSB-474-374-857,5375,100000,In-Person,Q3,DSB
5,DSB-448-546-348,4525,100009,Online,Q2,DSB
11,DSB-422-218-322,118,100010,Online,Q4,DSB
12,DSB-669-227-170,830,100001,Online,Q2,DSB


In [60]:
transactions = transactions.groupby(['Online or In-Person', 'Transaction Date']).agg({'Value':'sum'}).reset_index().rename(columns={'Transaction Date':'Quarter'})
transactions

,Online or In-Person,Quarter,Value
0,In-Person,Q1,47115
1,In-Person,Q2,81627
2,In-Person,Q3,84048
3,In-Person,Q4,52832
4,Online,Q1,55870
5,Online,Q2,94860
6,Online,Q3,56617
7,Online,Q4,57520


In [61]:
targets = pd.melt(targets, id_vars='Online or In-Person', var_name='Quarter', value_name='Targets')
targets

,Online or In-Person,Quarter,Targets
0,Online,Q1,72500
1,In-Person,Q1,75000
2,Online,Q2,70000
3,In-Person,Q2,70000
4,Online,Q3,60000
5,In-Person,Q3,70000
6,Online,Q4,60000
7,In-Person,Q4,60000


In [62]:
df = pd.merge(transactions, targets, on = ['Online or In-Person', 'Quarter'])
df['Variance in Targets'] = df['Value'] - df['Targets']

In [63]:
df

,Online or In-Person,Quarter,Value,Targets,Variance in Targets
0,In-Person,Q1,47115,75000,-27885
1,In-Person,Q2,81627,70000,11627
2,In-Person,Q3,84048,70000,14048
3,In-Person,Q4,52832,60000,-7168
4,Online,Q1,55870,72500,-16630
5,Online,Q2,94860,70000,24860
6,Online,Q3,56617,60000,-3383
7,Online,Q4,57520,60000,-2480
